In [10]:
import rasterio
import geopandas as gpd
from shapely.geometry import mapping
import json


clay_raster = rasterio.open('./soil_data/clay.tif')
sand_raster = rasterio.open('./soil_data/sand.tif')
density_raster = rasterio.open('./soil_data/density.tif')

fields = gpd.read_file('./field_centroids.geojson')

moisture_data = rasterio.open('../soil_moisture.tif')

field_list = []

for index, row in fields.iterrows():
    field_id = row['id']
    field_name = row['Name']
    centroid = row['geometry'].centroid
    lat, lon = centroid.y, centroid.x
    row, col = clay_raster.index(lon, lat)

    clay = clay_raster.read(1)[row, col]
    sand = sand_raster.read(1)[row, col]
    density = density_raster.read(1)[row, col]

    row, col = sand_raster.index(lon, lat)

    moisture = moisture_data.read(1)[row, col]

    field_dict = {
        'type': 'Feature',
        'properties': {
            'id': field_id,
            'name': field_name,
            'soil': {
                'clay': float(clay),
                'sand': float(sand),
                'density': int(density)
            },
            'soil_moisture': int(moisture),
            'coordinates': {
                'lat': float(lat),
                'lng': float(lon)
            },
            'geometry': mapping(centroid)
        },
    }

    field_list.append(field_dict)

with open('field_result.json', 'w', encoding='utf-8') as f:
    json.dump(field_list, f, ensure_ascii=False, indent=4)


Натхнення брав тут - https://github.com/KharchenkoV/geo/tree/master/lab6